In [1]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash 
pwd

/content/Template_Project


In [3]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [4]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"])
train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)

In [7]:
learn = Learner(train_ds, setup_config)
learn.fit(4)
# a = iter(learn.data.train_dl)
# b, c = next(a)
# learn.learn.model(learn.learn.data.train_ds.x.shape[1], 10)
# next(learn.learn.model.parameters()).device


100%|██████████| 750/750 [00:00<00:00, 342783.92it/s]


In [8]:
next(learn.learn.model.parameters()).device

device(type='cuda', index=0)

In [16]:
class test1:
    def __init__(self, a):
        self.a = a
    def test_func(self):
        return self.a
a = test1(1)
setattr(a, "s", 123)
vars(a)

{'a': 1, 's': 123}

In [17]:
monitor = Monitor_Cb(["val_acc"])
early = EarlyStopping_Cb(monitor="val_loss")
recorder = Recorder_Cb("hi")
cbh = CallbackHandler([monitor, early, recorder])
cbh.Monitor_Cb.learn


NameError: name 'Monitor_Cb' is not defined

In [9]:
a = ["a"]
for i in a[:4]:
    print(i)

a
